In [10]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

In [2]:
df_groundtruth = pd.read_csv('../ClaimBuster_Datasets/datasets/groundtruth.csv')
df_all_sentences = pd.read_csv('../ClaimBuster_Datasets/datasets/all_sentences.csv')
df_crowdsourced = pd.read_csv('../ClaimBuster_Datasets/datasets/crowdsourced.csv')
df_crwd_grnd = pd.concat([df_groundtruth,df_crowdsourced],axis=0)
sentence_id_with_v = df_crwd_grnd['Sentence_id'].values
df_without_label = df_all_sentences[~((df_all_sentences['Speaker'] == 'Information') | (df_all_sentences['Speaker_title'].isna()) | (df_all_sentences['Sentence_id'].isin(sentence_id_with_v))) ]
df_without_label['Verdict'] = 7 #all non labels have 7 as verdict
df_full = pd.concat([df_crwd_grnd,df_without_label],axis=0)
df_full = df_full.sort_values('Sentence_id')
df_full.reset_index(drop=True,inplace=True)

# df_crwd_grnd.sort_values('Sentence_id').head()


/var/folders/x7/tvh019v93cx4qhm55ttc6yp80000gn/T/ipykernel_80979/2084367091.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_label['Verdict'] = 7 #all non labels have 7 as verdict


In [3]:
def generate_word_dataset(df,presentence = 1,postsentence = 1):
    dataset = []
    label = []
    for index,row in df.iterrows():
        if index == 0 or index == len(df)-1:
            continue
        past = df.iloc[index-1]['Text']
        curr = row['Text']
        future = df.iloc[index+1]['Text']    
        if row['File_id'] != df.iloc[index-1]['File_id']:
            past = "EMPTY"
        if row['File_id'] != df.iloc[index+1]['File_id']:
            future = "EMPTY"
        dataset.append([past,curr,future])
        label.append(row['Verdict'])
    
    return dataset,label

data_text, labels = generate_word_dataset(df_full)

In [11]:
class PreprocessedData(Dataset):
    def __init__(self, pseudolabels = False, num_context = 1):
        super().__init__()
        self.df = self.extract()
        self.sentences, self.labels = self.generate_word_dataset(df=self.df, num_context=num_context, pseudolabels = pseudolabels)

    def __len__(self):
            return len(self.labels)
    
    def __getitem__(self, idx):
            label = self.labels[idx]
            sentences = self.sentences[idx]
            sample = {"Sentences": sentences, "Class": label}
            return sample
    
    def extract(self):
        df_groundtruth = pd.read_csv('../ClaimBuster_Datasets/datasets/groundtruth.csv')
        df_all_sentences = pd.read_csv('../ClaimBuster_Datasets/datasets/all_sentences.csv')
        df_crowdsourced = pd.read_csv('../ClaimBuster_Datasets/datasets/crowdsourced.csv')
        df_crwd_grnd = pd.concat([df_groundtruth,df_crowdsourced],axis=0)
        sentence_id_with_v = df_crwd_grnd['Sentence_id'].values
        df_without_label = df_all_sentences[~((df_all_sentences['Speaker'] == 'Information') | (df_all_sentences['Speaker_title'].isna()) | (df_all_sentences['Sentence_id'].isin(sentence_id_with_v))) ]
        df_without_label['Verdict'] = 7 #all non labels have 7 as verdict
        df_full = pd.concat([df_crwd_grnd,df_without_label],axis=0)
        df_full = df_full.sort_values('Sentence_id')
        df_full.reset_index(drop=True,inplace=True)
        return df_full
    
    def generate_word_dataset(self,df,num_context,pseudolabels):
        dataset = []
        label = []
        for index,row in df.iterrows():
            if index == 0 or index == len(df)-num_context:
                continue
            past = df.iloc[index-1]['Text']
            curr = row['Text']
            future = df.iloc[index+1]['Text']    
            if row['File_id'] != df.iloc[index-1]['File_id']:
                past = "EMPTY"
            if row['File_id'] != df.iloc[index+1]['File_id']:
                future = "EMPTY"
            dataset.append([past,curr,future])
            label.append(row['Verdict'])
        
        label = np.array(label)

        labelled_indices = np.where(label!=7)[0]
        label = label.tolist()

        if pseudolabels == False:
            dataset = dataset[labelled_indices]
            label = label[labelled_indices]

        return dataset,label
    